# Integration File
In this integration file, we performed the initial merging of all data sources with only minimal preprocessing. First, we linked the location dataset with the wildfire dataset to assign each fire event to its correct city, ensuring that wildfire records can later be aligned with weather and air-quality data. Next, we merged the weather dataset with the PM2.5 dataset, since they share identical city names and the same daily time range. After preparing both parts, we combined the integrated weather–PM data with the wildfire data to form a single unified dataset. At this stage, a few issues remain—such as duplicates, missing values, and potential outliers—which will be handled in the subsequent cleaning and feature-engineering steps.

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# WildFire

In [ ]:
fires=pd.read_csv('https://github.com/MajdBa7r/AQI/raw/main/Fire_Disturbance_Point%20(1).csv')

In [ ]:
fires

,X,Y,OGF_ID,FIRE_DISTURBANCE_AREA_IDENT,FIRE_TYPE_CODE,FIRE_YEAR,FIRE_GENERAL_CAUSE_CODE,FIRE_WEATHER_INDEX,FIRE_RESPONSE_OBJ_CODE,FIRE_START_DATE,FIRE_OUT_DATE,FIRE_FINAL_SIZE,BUSINESS_EFFECTIVE_DATE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID
0,-88.0381,49.0228,43129190,NIP2,IFR,2003,RWY,0.000000,SUP,2003/04/13 00:00:00+00,2003/04/14 00:00:00+00,1.5,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,1
1,-86.7287,49.8475,43129191,NIP20,IFR,2003,UNK,16.200001,SUP,2003/05/27 00:00:00+00,2003/05/27 00:00:00+00,0.1,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,2
2,-88.0725,50.2296,43129192,NIP21,IFR,2003,RWY,19.200001,SUP,2003/05/25 00:00:00+00,2003/05/25 00:00:00+00,15.0,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,3
3,-88.5994,53.4975,43129193,NIP22,IFR,2003,REC,26.900000,SUP,2003/05/27 00:00:00+00,2003/05/28 00:00:00+00,0.4,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,4
4,-87.9607,50.2486,43129194,NIP23,IFR,2003,RWY,19.200001,SUP,2003/05/25 00:00:00+00,2003/05/25 00:00:00+00,0.2,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65001,-90.0352,51.4709,320961613,SLK9,IFR,2024,LTG,14.000000,FUL,2024/07/07 00:00:00+00,2024/07/09 00:00:00+00,0.6,NaN,2025/02/21 11:34:46+00,2025/02/21 10:58:59+00,2025/02/21 22:19:59+00,938657
65002,-93.2444,52.4091,320961347,RED3,IFR,2024,LTG,8.000000,FUL,2024/07/06 00:00:00+00,2024/07/12 00:00:00+00,4.8,NaN,2025/02/21 11:34:46+00,2025/02/21 10:58:33+00,2025/02/21 22:19:59+00,938673
65003,-85.6377,48.6897,320961181,WAW15,IFR,2024,UNK,0.000000,FUL,2024/08/28 00:00:00+00,2024/08/28 00:00:00+00,0.1,NaN,2025/02/21 11:34:47+00,2025/02/21 10:58:16+00,2025/02/21 22:19:59+00,938689
65004,-94.5996,49.8023,320961430,KEN9,IFR,2024,UNK,0.000000,FUL,2024/06/03 00:00:00+00,2024/06/04 00:00:00+00,0.1,NaN,2025/02/21 11:34:42+00,2025/02/21 10:58:42+00,2025/02/21 22:19:59+00,938705


In [ ]:
fires['longitude']=fires['X']
fires['latitude']=fires['Y']
fires.drop(['X','Y'],axis=1,inplace=True)#only for unifying names

In [ ]:
locations=pd.read_csv('https://github.com/LamaEmran/Data/raw/main/ontario_wildfires_refined.csv')#locations file

In [ ]:
locations

,longitude,latitude,Province,City / Town
0,-88.0381,49.0228,Ontario,Greenstone
1,-86.7287,49.8475,Ontario,Greenstone
2,-88.0725,50.2296,Ontario,Greenstone
3,-88.5994,53.4975,Ontario,Greenstone
4,-87.9607,50.2486,Ontario,Greenstone
...,...,...,...,...
58260,-84.7862,48.4938,Ontario,Hornepayne
58261,-90.0352,51.4709,Ontario,Atikokan
58262,-93.2444,52.4091,Ontario,Red Lake
58263,-85.6377,48.6897,Ontario,Marathon


In [ ]:
g1=gpd.GeoDataFrame(fires, geometry=gpd.points_from_xy(fires['longitude'], fires['latitude']))
g2=gpd.GeoDataFrame(locations, geometry=gpd.points_from_xy(locations['longitude'] , locations['latitude']))

In [ ]:
g1.crs = "EPSG:4326"
g2.crs = "EPSG:4326"
g1_meters = g1.to_crs("EPSG:3347")#EPSG:3347 is the Statistics Canada Lambert projection (units = meters)
g2_meters = g2.to_crs("EPSG:3347")

We converted both datasets from geographic coordinates (EPSG:4326) to a projected coordinate system (EPSG:3347) so that distances are measured in meters. This is necessary because spatial operations like sjoin_nearest and max_distance require accurate linear distances, which cannot be computed correctly in latitude–longitude

If we don’t convert to a projected coordinate system, distance calculations would be inaccurate because latitude–longitude degrees are not uniform in length. As a result, spatial joins using max_distance or nearest-neighbor methods **could assign fires to the wrong cities or fail to find nearby points.**

In [ ]:
merged = g2_meters.sjoin_nearest(g1_meters, how='inner', max_distance=100000)

We used sjoin_nearest instead of an exact merge because wildfire points rarely match location coordinates exactly, so nearest-neighbor spatial joining is the correct way to assign each fire to the closest city based on geographic distance.

In [ ]:
merged.duplicated().sum()

np.int64(30378)

In [ ]:
merged.drop_duplicates(inplace=True)

A nearest-neighbor spatial join (sjoin_nearest) produces duplicates because it matches each point to the closest geographic feature rather than enforcing a one-to-one relationship. When multiple observations share the same nearest wildfire point, that wildfire record is repeated for each observation. Since the method is based purely on spatial proximity—not on unique relational keys like city or date—duplicate matches naturally occur.

In [ ]:
merged

,longitude_left,latitude_left,Province,City / Town,geometry,index_right,OGF_ID,FIRE_DISTURBANCE_AREA_IDENT,FIRE_TYPE_CODE,FIRE_YEAR,...,FIRE_START_DATE,FIRE_OUT_DATE,FIRE_FINAL_SIZE,BUSINESS_EFFECTIVE_DATE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID,longitude_right,latitude_right
0,-88.0381,49.0228,Ontario,Greenstone,POINT (6479821.435 1439947.521),42,43129232,NIP60,IFR,2003,...,2003/06/29 00:00:00+00,2003/06/29 00:00:00+00,0.3,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,43,-88.0381,49.0228
0,-88.0381,49.0228,Ontario,Greenstone,POINT (6479821.435 1439947.521),0,43129190,NIP2,IFR,2003,...,2003/04/13 00:00:00+00,2003/04/14 00:00:00+00,1.5,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,1,-88.0381,49.0228
1,-86.7287,49.8475,Ontario,Greenstone,POINT (6567952.169 1537962.442),1,43129191,NIP20,IFR,2003,...,2003/05/27 00:00:00+00,2003/05/27 00:00:00+00,0.1,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,2,-86.7287,49.8475
2,-88.0725,50.2296,Ontario,Greenstone,POINT (6469327.381 1573463.803),2,43129192,NIP21,IFR,2003,...,2003/05/25 00:00:00+00,2003/05/25 00:00:00+00,15.0,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,3,-88.0725,50.2296
3,-88.5994,53.4975,Ontario,Greenstone,POINT (6413478.976 1930876.384),3,43129193,NIP22,IFR,2003,...,2003/05/27 00:00:00+00,2003/05/28 00:00:00+00,0.4,2004/06/16 00:00:00+00,NaN,2004/06/16 00:00:00+00,2005/04/22 08:35:11+00,4,-88.5994,53.4975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58260,-84.7862,48.4938,Ontario,Hornepayne,POINT (6723280.643 1401795.575),65000,320961228,WAW13,IFR,2024,...,2024/08/21 00:00:00+00,2024/09/13 00:00:00+00,29.5,NaN,2025/02/21 11:34:47+00,2025/02/21 10:58:21+00,2025/02/21 22:19:59+00,938641,-84.7862,48.4938
58261,-90.0352,51.4709,Ontario,Atikokan,POINT (6326116.319 1704402.146),65001,320961613,SLK9,IFR,2024,...,2024/07/07 00:00:00+00,2024/07/09 00:00:00+00,0.6,NaN,2025/02/21 11:34:46+00,2025/02/21 10:58:59+00,2025/02/21 22:19:59+00,938657,-90.0352,51.4709
58262,-93.2444,52.4091,Ontario,Red Lake,POINT (6107360.001 1806899.561),65002,320961347,RED3,IFR,2024,...,2024/07/06 00:00:00+00,2024/07/12 00:00:00+00,4.8,NaN,2025/02/21 11:34:46+00,2025/02/21 10:58:33+00,2025/02/21 22:19:59+00,938673,-93.2444,52.4091
58263,-85.6377,48.6897,Ontario,Marathon,POINT (6658432.456 1416924.865),65003,320961181,WAW15,IFR,2024,...,2024/08/28 00:00:00+00,2024/08/28 00:00:00+00,0.1,NaN,2025/02/21 11:34:47+00,2025/02/21 10:58:16+00,2025/02/21 22:19:59+00,938689,-85.6377,48.6897


In [ ]:
merged['City / Town'].value_counts().shape


(73,)

In [ ]:
merged['Province'].value_counts().shape


(1,)

In [ ]:
y=merged[merged.FIRE_YEAR.isin([2017,2018,2019,2020,2021,2022,2023,2024])].copy() #filter the merged wildfire dataset to keep only records from years 2017 to 2024

In [ ]:
y['City / Town'].value_counts().shape

(47,)

In [ ]:
city_names = y['City / Town'].unique()

In [ ]:
city_names

array(['Atikokan', 'Red Lake', 'Thunder Bay', 'Dryden', 'Greenstone',
       'Ear Falls', 'Hornepayne', 'Kapuskasing', 'Temiskaming Shores',
       'Hearst', 'Sault Ste. Marie', 'Deep River', 'Rayside-Balfour',
       'Greater Sudbury', 'Marathon', 'Espanola', 'Pembroke',
       'Elliot Lake', 'Fort Frances', 'Iroquois Falls', 'Englehart',
       'South River', 'Kirkland Lake', 'Neebing', 'Huntsville', 'Mattawa',
       'Peterborough', 'Timmins', 'Parry Sound', 'Gravenhurst', 'Midland',
       'North Bay', 'Bancroft', 'Renfrew', 'Skatepark', 'Omemee',
       'Thessalon', 'Tobermory', 'Powassan', 'Perth', 'Greater Napanee',
       'Little Current', 'Bracebridge', 'Belleville', 'Arnprior',
       'Petawawa', 'Orillia'], dtype=object)

In [ ]:
y.to_csv('ontario_wildfires_from2017to2024.csv',index=False)

In [ ]:
#this is only to check against the previous integration method we applied
NF= pd.read_csv('https://raw.githubusercontent.com/frxqie/Data_Eng_Data/main/ontario_wildfires_prepped.csv')
NF=NF[NF.FIRE_YEAR.isin([2017,2018,2019,2020,2021,2022,2023,2024])]

In [ ]:
NF['City / Town'].value_counts().shape

(46,)

In [ ]:
y['City / Town'].value_counts().shape

(47,)

In [ ]:
cities1 = set(y['City / Town'].str.strip().str.lower())
cities2 = set(NF['City / Town'].str.strip().str.lower())

common = cities1 & cities2
missing_from_df2 = cities1 - cities2
missing_from_df1 = cities2 - cities1

print("Common cities:", len(common))
print("Cities missing from Naser and Farah cities:", missing_from_df2)
print("Cities missing from our cities:", missing_from_df1)

Common cities: 46
Cities missing from Naser and Farah cities: {'skatepark'}
Cities missing from our cities: set()


In [ ]:
y[y['City / Town'].str.lower().str.contains('skatepark')]

,longitude_left,latitude_left,Province,City / Town,geometry,index_right,OGF_ID,FIRE_DISTURBANCE_AREA_IDENT,FIRE_TYPE_CODE,FIRE_YEAR,...,FIRE_START_DATE,FIRE_OUT_DATE,FIRE_FINAL_SIZE,BUSINESS_EFFECTIVE_DATE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID,longitude_right,latitude_right
50946,-77.0506,44.8843,Ontario,Skatepark,POINT (7380871.045 1105644.867),56962,125435708,BAN15,IFR,2018,...,2018/07/05 00:00:00+00,2018/07/09 00:00:00+00,0.1,2019/03/22 11:01:56+00,2019/03/22 11:06:23+00,2019/03/22 11:02:15+00,2019/03/22 21:57:15+00,771633,-77.0506,44.8843
51142,-76.9863,44.6325,Ontario,Skatepark,POINT (7392509.675 1079114.897),57187,125435897,PET3,IFR,2018,...,2018/07/12 00:00:00+00,2018/07/20 00:00:00+00,4.3,2019/03/22 11:01:56+00,2019/03/22 11:06:26+00,2019/03/22 11:02:19+00,2019/03/22 21:57:15+00,775233,-76.9863,44.6325
51667,-76.8204,44.6920,Ontario,Skatepark,POINT (7403978.245 1088792.775),57775,125434978,PET1,IFR,2018,...,2018/05/14 00:00:00+00,2018/05/15 00:00:00+00,0.8,2019/03/22 11:01:56+00,2019/03/22 11:06:14+00,2019/03/22 11:02:16+00,2019/03/22 21:57:15+00,784641,-76.8204,44.6920
52002,-76.7849,44.5344,Ontario,Skatepark,POINT (7410958.828 1072113.285),58160,127799207,PET3,IFR,2019,...,2019/08/01 00:00:00+00,2019/08/27 00:00:00+00,4.2,2020/02/20 14:45:15+00,2020/02/20 14:48:49+00,2020/02/20 14:45:31+00,2020/02/20 21:47:15+00,791041,-76.7849,44.5344
53736,-76.9254,44.7786,Ontario,Skatepark,POINT (7393450.161 1096346.632),60140,165649128,PET1,IFR,2021,...,2021/06/13 00:00:00+00,2021/06/22 00:00:00+00,6.0,2022/03/07 11:44:04+00,2022/03/07 11:52:58+00,2022/03/07 11:44:27+00,2022/03/09 04:09:25+00,823041,-76.9254,44.7786


In [ ]:
y['City / Town'] = y['City / Town'].replace('Skatepark', 'Frontenac')

Manually checked the locations of these entries on maps and corrected them accordingly

In [ ]:
y["FIRE_START_DATE"] = pd.to_datetime(
    y["FIRE_START_DATE"],
    errors="coerce"
).dt.date

y["FIRE_OUT_DATE"] = pd.to_datetime(
    y["FIRE_OUT_DATE"],
    errors="coerce"
).dt.date
#This ensures consistent date handling

In [ ]:
y['date']=y['FIRE_START_DATE']
y['city']=y['City / Town']

In [ ]:
y['city'] = y['city'].str.strip().str.lower()
y['date'] = pd.to_datetime(y['date'])

In [ ]:
y.columns

Index(['longitude_left', 'latitude_left', 'Province', 'City / Town',
       'geometry', 'index_right', 'OGF_ID', 'FIRE_DISTURBANCE_AREA_IDENT',
       'FIRE_TYPE_CODE', 'FIRE_YEAR', 'FIRE_GENERAL_CAUSE_CODE',
       'FIRE_WEATHER_INDEX', 'FIRE_RESPONSE_OBJ_CODE', 'FIRE_START_DATE',
       'FIRE_OUT_DATE', 'FIRE_FINAL_SIZE', 'BUSINESS_EFFECTIVE_DATE',
       'GEOMETRY_UPDATE_DATETIME', 'EFFECTIVE_DATETIME', 'SYSTEM_DATETIME',
       'OBJECTID', 'longitude_right', 'latitude_right', 'date', 'city'],
      dtype='object')

In [ ]:
y[['date','city']].duplicated().sum()

np.int64(2168)

Duplicates appear when merging by date and city because multiple wildfire events can occur in the same city on the same day. Each fire becomes a separate row, so when the dataset is merged with daily city-level data, one day can match multiple fire records. Aggregating the wildfire dataset by date and city (summing fire size or averaging fire indices) resolves this by producing a single consolidated record per day per city.

In [ ]:
mask =y['FIRE_OUT_DATE'] < y['FIRE_START_DATE']
y.loc[mask, ['FIRE_START_DATE', 'FIRE_OUT_DATE']]=y.loc[mask, ['FIRE_OUT_DATE', 'FIRE_START_DATE']].values

# Weather and PM

In [ ]:
weather_url = "https://github.com/LamaEmran/Data/raw/main/All_Cities_Weather_Final.csv"
weather = pd.read_csv(weather_url)
aqi = pd.read_csv('https://github.com/MajdBa7r/AQI/raw/main/PM2.5.csv')

These files have already been prepared for integration in other notebooks

In [ ]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270322 entries, 0 to 270321
Data columns (total 19 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   time                               270322 non-null  object 
 1   precipitation_sum (mm)             270322 non-null  float64
 2   rain_sum (mm)                      270322 non-null  float64
 3   snowfall_sum (cm)                  270322 non-null  float64
 4   relative_humidity_2m_mean (%)      270322 non-null  int64  
 5   relative_humidity_2m_max (%)       270322 non-null  int64  
 6   relative_humidity_2m_min (%)       270322 non-null  int64  
 7   dew_point_2m_mean (°C)             270322 non-null  float64
 8   shortwave_radiation_sum (MJ/m²)    270322 non-null  float64
 9   sunshine_duration (s)              270321 non-null  float64
 10  temperature_2m_mean (°C)           270322 non-null  float64
 11  temperature_2m_max (°C)            2703

In [ ]:
weather['city']=weather['City']
weather['date']=weather['time']

In [ ]:
weather

,time,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),relative_humidity_2m_mean (%),relative_humidity_2m_max (%),relative_humidity_2m_min (%),dew_point_2m_mean (°C),shortwave_radiation_sum (MJ/m²),sunshine_duration (s),...,temperature_2m_max (°C),temperature_2m_min (°C),wind_speed_10m_mean (km/h),wind_speed_10m_max (km/h),winddirection_10m_dominant (°),vapour_pressure_deficit_max (kPa),et0_fao_evapotranspiration (mm),City,city,date
0,2015-01-01,0.0,0.0,0.00,47,57,24,-13.0,7.36,27754.97,...,5.8,-9.7,6.1,8.4,9,0.70,1.00,Angus,Angus,2015-01-01
1,2015-01-02,0.0,0.0,0.00,50,61,34,-9.0,4.78,14817.90,...,7.4,-5.4,4.6,7.6,10,0.68,0.78,Angus,Angus,2015-01-02
2,2015-01-03,2.7,0.0,2.10,79,89,65,-7.1,4.19,15319.11,...,-1.5,-8.0,6.1,7.9,107,0.18,0.39,Angus,Angus,2015-01-03
3,2015-01-04,0.0,0.0,0.00,60,67,43,-13.9,7.63,27817.25,...,1.5,-13.0,5.9,7.2,17,0.36,0.76,Angus,Angus,2015-01-04
4,2015-01-05,0.0,0.0,0.00,48,66,30,-13.1,5.57,18155.53,...,4.0,-12.9,5.1,7.9,357,0.57,0.81,Angus,Angus,2015-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270317,2024-12-27,0.0,0.0,0.00,88,90,84,-1.1,4.33,11162.59,...,2.6,-0.9,20.8,24.3,136,0.11,0.44,Wingham,Wingham,2024-12-27
270318,2024-12-28,1.6,1.6,0.00,91,99,79,4.3,4.73,19087.58,...,11.2,1.6,23.4,30.8,174,0.27,0.57,Wingham,Wingham,2024-12-28
270319,2024-12-29,21.4,21.4,0.00,97,99,92,3.7,0.69,0.00,...,6.7,2.1,12.4,20.3,90,0.08,0.16,Wingham,Wingham,2024-12-29
270320,2024-12-30,15.4,8.6,4.76,98,100,93,2.8,1.32,0.00,...,7.8,0.7,22.6,31.0,266,0.05,0.14,Wingham,Wingham,2024-12-30


In [ ]:
aqi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216228 entries, 0 to 216227
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   city    216228 non-null  object 
 1   date    216228 non-null  object 
 2   lat     216228 non-null  float64
 3   lon     216228 non-null  float64
 4   pm25    216228 non-null  float64
 5   year    216228 non-null  int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 9.9+ MB


In [ ]:
aqi

,city,date,lat,lon,pm25,year
0,Angus,2017-01-01,44.1552,-79.8932,11.076371,2017
1,Arnprior,2017-01-01,45.2001,-76.5278,2.090151,2017
2,Atikokan,2017-01-01,51.2216,-90.2304,0.128585,2017
3,Bancroft,2017-01-01,45.5890,-78.3437,2.089787,2017
4,Barrie,2017-01-01,44.4243,-79.7877,10.341466,2017
...,...,...,...,...,...,...
216223,Timmins,2024-12-31,49.1549,-81.2948,0.055960,2024
216224,Tobermory,2024-12-31,45.6419,-81.7172,0.211195,2024
216225,Wasaga Beach,2024-12-31,44.4896,-80.0253,0.770121,2024
216226,Welland,2024-12-31,42.9326,-79.3149,1.562177,2024


In [ ]:
merged2 = aqi.merge(weather, on=['date','city'], how='left')

In [ ]:
merged2['city'] = merged2['city'].replace('Skatepark', 'Frontenac')

In [ ]:
merged2.to_csv('weather and PM.csv',index=False)

In [ ]:
merged2['city'].value_counts().shape

(74,)

In [ ]:
city_names=y['City / Town'].unique()
city_names

array(['Atikokan', 'Red Lake', 'Thunder Bay', 'Dryden', 'Greenstone',
       'Ear Falls', 'Hornepayne', 'Kapuskasing', 'Temiskaming Shores',
       'Hearst', 'Sault Ste. Marie', 'Deep River', 'Rayside-Balfour',
       'Greater Sudbury', 'Marathon', 'Espanola', 'Pembroke',
       'Elliot Lake', 'Fort Frances', 'Iroquois Falls', 'Englehart',
       'South River', 'Kirkland Lake', 'Neebing', 'Huntsville', 'Mattawa',
       'Peterborough', 'Timmins', 'Parry Sound', 'Gravenhurst', 'Midland',
       'North Bay', 'Bancroft', 'Renfrew', 'Frontenac', 'Omemee',
       'Thessalon', 'Tobermory', 'Powassan', 'Perth', 'Greater Napanee',
       'Little Current', 'Bracebridge', 'Belleville', 'Arnprior',
       'Petawawa', 'Orillia'], dtype=object)

In [ ]:
filtered = merged2[merged2['city'].isin(city_names)].copy()

In [ ]:
filtered['city'].value_counts().shape

(47,)

In [ ]:
filtered.duplicated().sum()

np.int64(0)

In [ ]:
filtered['city'] = filtered['city'].str.strip().str.lower()
filtered['date'] = pd.to_datetime(filtered['date'])

Standardize city names and convert dates to datetime format for consistent merging and analysis

# Full Integration

In [ ]:
final_merged = filtered.merge(y,on=['date','city'],how='left',indicator=True)

In [ ]:
final_merged

,city,date,lat,lon,pm25,year,time,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),...,FIRE_OUT_DATE,FIRE_FINAL_SIZE,BUSINESS_EFFECTIVE_DATE,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID,longitude_right,latitude_right,_merge
0,arnprior,2017-01-01,45.2001,-76.5278,2.090151,2017,2017-01-01,2.8,0.0,1.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,atikokan,2017-01-01,51.2216,-90.2304,0.128585,2017,2017-01-01,0.9,0.0,0.63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,bancroft,2017-01-01,45.5890,-78.3437,2.089787,2017,2017-01-01,1.9,0.0,1.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,belleville,2017-01-01,44.4855,-77.4459,4.860390,2017,2017-01-01,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,bracebridge,2017-01-01,45.1669,-79.2325,6.356419,2017,2017-01-01,1.8,0.2,1.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139497,temiskaming shores,2024-12-31,47.1689,-79.7346,0.086636,2024,2024-12-31,0.7,0.1,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
139498,thessalon,2024-12-31,46.4957,-83.7170,0.260759,2024,2024-12-31,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
139499,thunder bay,2024-12-31,49.1197,-88.4047,0.033853,2024,2024-12-31,0.0,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
139500,timmins,2024-12-31,49.1549,-81.2948,0.055960,2024,2024-12-31,0.2,0.0,0.14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [ ]:
final_merged['_merge'].value_counts()

,count
_merge,
left_only,134160
both,5342
right_only,0


In [ ]:
final_merged.duplicated().sum()

np.int64(0)

In [ ]:
final_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139502 entries, 0 to 139501
Data columns (total 49 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   city                               139502 non-null  object        
 1   date                               139502 non-null  datetime64[ns]
 2   lat                                139502 non-null  float64       
 3   lon                                139502 non-null  float64       
 4   pm25                               139502 non-null  float64       
 5   year                               139502 non-null  int64         
 6   time                               139502 non-null  object        
 7   precipitation_sum (mm)             139502 non-null  float64       
 8   rain_sum (mm)                      139502 non-null  float64       
 9   snowfall_sum (cm)                  139502 non-null  float64       
 10  relative_humidity_2m

In [ ]:
final_merged[final_merged['FIRE_FINAL_SIZE']==1][:2].T


,4887,5926
city,thunder bay,atikokan
date,2017-04-14 00:00:00,2017-05-07 00:00:00
lat,49.1197,51.2216
lon,-88.4047,-90.2304
pm25,1.995694,0.698778
year,2017,2017
time,2017-04-14,2017-05-07
precipitation_sum (mm),0.0,0.0
rain_sum (mm),0.0,0.0
snowfall_sum (cm),0.0,0.0


In [ ]:
final_merged['City / Town'].value_counts().shape

(47,)

In [ ]:
final_merged['City'].value_counts().shape

(47,)

In [ ]:
cols_to_drop=['Province', 'City', 'City / Town', 'longitude_left', 'latitude_left', 'longitude_right', 'latitude_right', 'OBJECTID','time',
            'FIRE_DISTURBANCE_AREA_IDENT','BUSINESS_EFFECTIVE_DATE', 'GEOMETRY_UPDATE_DATETIME','EFFECTIVE_DATETIME', 'SYSTEM_DATETIME','_merge']

In [ ]:
final_merged.drop(cols_to_drop,axis=1,inplace=True)

In [ ]:
final_merged

,city,date,lat,lon,pm25,year,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),relative_humidity_2m_mean (%),...,index_right,OGF_ID,FIRE_TYPE_CODE,FIRE_YEAR,FIRE_GENERAL_CAUSE_CODE,FIRE_WEATHER_INDEX,FIRE_RESPONSE_OBJ_CODE,FIRE_START_DATE,FIRE_OUT_DATE,FIRE_FINAL_SIZE
0,arnprior,2017-01-01,45.2001,-76.5278,2.090151,2017,2.8,0.0,1.96,80.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,atikokan,2017-01-01,51.2216,-90.2304,0.128585,2017,0.9,0.0,0.63,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bancroft,2017-01-01,45.5890,-78.3437,2.089787,2017,1.9,0.0,1.33,82.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,belleville,2017-01-01,44.4855,-77.4459,4.860390,2017,0.0,0.0,0.00,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bracebridge,2017-01-01,45.1669,-79.2325,6.356419,2017,1.8,0.2,1.12,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139497,temiskaming shores,2024-12-31,47.1689,-79.7346,0.086636,2024,0.7,0.1,0.42,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139498,thessalon,2024-12-31,46.4957,-83.7170,0.260759,2024,0.0,0.0,0.00,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139499,thunder bay,2024-12-31,49.1197,-88.4047,0.033853,2024,0.0,0.0,0.00,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139500,timmins,2024-12-31,49.1549,-81.2948,0.055960,2024,0.2,0.0,0.14,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139502 entries, 0 to 139501
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   city                               139502 non-null  object        
 1   date                               139502 non-null  datetime64[ns]
 2   lat                                139502 non-null  float64       
 3   lon                                139502 non-null  float64       
 4   pm25                               139502 non-null  float64       
 5   year                               139502 non-null  int64         
 6   precipitation_sum (mm)             139502 non-null  float64       
 7   rain_sum (mm)                      139502 non-null  float64       
 8   snowfall_sum (cm)                  139502 non-null  float64       
 9   relative_humidity_2m_mean (%)      139502 non-null  float64       
 10  relative_humidity_2m

In [ ]:
final_merged[['date','city']].duplicated().sum()

np.int64(2168)

In [ ]:
final_merged.to_csv('ontario_wildfire_weather_pm_2017_2024_2.csv',index=False)